In [2]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Особенности Си

Сегодня в программе:
* <a href="#primitives" style="color:#856024"> Примитивные типы и их вывод </a>
* <a href="#structures" style="color:#856024"> Структуры </a>
* <a href="#arrays" style="color:#856024"> Массивы </a>
* <a href="#dyn_arrays" style="color:#856024"> Динамические массивы </a>

[Пиши на C как джентльмен / Хабр](https://habr.com/ru/post/325678/)

[Здесь](/sem02-instruments-compilation) в конце (дополнение про макросы) есть больше примеров про макросы, если интересно

## <a name="primitives"></a> Примитивные типы и их вывод

In [1]:
%%cpp main.c
%run gcc -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe

#include <stdio.h>

int main() {
    int i = 5;
    float f = 5.5555555555555555555555555;
    double d = 5.5555555555555555555555555;
    char c = 'X';
    char* str = "Hello world!";
    printf("i = %d\n", i);
    printf("f = %f, f = %0.20f\n", f, f); // обратите внимание на точность
    printf("d = %lf, d = %0.20lf\n", d, d);
    printf("c = %c, int(c) = %d\n", c, (int)c); // приведение типов
    printf("s = %s\n", str); // zero-terminated string
    printf("s = %.*s\n", 4, str); // string by begin and length
    printf("*str = %c, *(str + 1) = %c, str[1] = %c\n", *str, *(str + 1), str[1]);
    return 0;
}

UsageError: Cell magic `%%cpp` not found.


In [139]:
%%cpp main.c
%run gcc  -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe

#include <stdio.h>
#include <inttypes.h>


typedef int64_t i64;

int main() {
    int i = 5;
    int32_t i_32 = 6; // -2^31 до (2^31) - 1
    i64 i_64 = 7; // -2^63 до (2^63) - 1
    // uint64_t, int8_t, ...
    
    printf("i = %" "d" ", sizeof(i) = %d\n", i, (int)sizeof(i));
    //               [1]                                 [2]                        
    printf("i_32 = %" PRId32 ", sizeof(i_32) = %d\n", i_32, (int)sizeof(i_32));
    printf("i_64 = %" PRId64 ", sizeof(i_64) = %d\n", i_64, (int)sizeof(i_64));
    // [1] - правильный спецификатор
    // [2] - каст к типу спецификатора
    return 0;
}

Run: `gcc  -std=c99 -Wall -Werror main.c -o a.exe`

Run: `./a.exe`

i = 5, sizeof(i) = 4
i_32 = 6, sizeof(i_32) = 4
i_64 = 7, sizeof(i_64) = 8


## <a name="structures"></a> Структуры

In [140]:
%%cpp main.c
%run gcc -m64 -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe

#include <stdio.h>

struct point {
    double x, y;
};


struct point point_sum(struct point a, struct point b) {
    struct point c;
    c.x = a.x + b.x;
    c.y = a.y + b.y;
    return c;
} 


typedef struct point point_t;

point_t point_sub(point_t a, point_t b) {
    a.x -= b.x;
    a.y -= b.y;
    return a;
} 

point_t point_mul(point_t a, double k) {
    return (point_t){.x = a.x * k, .y = a.y * k};
}

void point_print(point_t a) {
    printf("{.x = %lf, .y = %lf}", a.x, a.y);
}

int main() {
    point_t a = {1, 100}, b = {3, 50};
    point_print(a); printf("\n");
    
    point_print(point_sum(a, b)); printf("\n");
    point_print(point_sub(a, b)); printf("\n");
    point_print(point_mul(a, 10)); printf("\n");
    
    return 0;
}

Run: `gcc -m64 -std=c99 -Wall -Werror main.c -o a.exe`

Run: `./a.exe`

{.x = 1.000000, .y = 100.000000}
{.x = 4.000000, .y = 150.000000}
{.x = -2.000000, .y = 50.000000}
{.x = 10.000000, .y = 1000.000000}


## <a name="arrays"></a> Массивы

In [141]:
%%cpp main.c
%run gcc -m64 -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe

#include <stdio.h>

#define CONST 2
#define SIZE(a) (sizeof(a) / sizeof((a)[0]))

int main() {
    char* s = "Hello wotrld!";
    printf("%s\n", s);
    char s1[] = "1";
    printf("%s\n", s1);
    char s2[100] = "Hello world! 2";
    printf("%s\n", s2);
    printf("%d %d %d\n", (int)sizeof(s), (int)sizeof(s1), (int)sizeof(s2));
        
    int a[] = {1, 5, 3, 2, 5};
    printf("sizeof(a) = %d\n", (int)sizeof(a));
    for (int i = 0; i < sizeof(a) / sizeof(int); ++i) {
        printf("a[%d] = %d, ", i, a[i]);
    }
    printf("\n");
    for (int i = 0; i < SIZE(a); ++i) {
        printf("a[%d] = %d, ", i, a[i]);
    }
    printf("\n");
        
    return 0;
}

Run: `gcc -m64 -std=c99 -Wall -Werror main.c -o a.exe`

Run: `./a.exe`

Hello wotrld!
1
Hello world! 2
8 2 100
sizeof(a) = 20
a[0] = 1, a[1] = 5, a[2] = 3, a[3] = 2, a[4] = 5, 
a[0] = 1, a[1] = 5, a[2] = 3, a[3] = 2, a[4] = 5, 


## <a name="dyn_arrays"></a> Динамические массивы

In [142]:
with open("001.in", "w") as f:
    f.write('''\
4
5 1 9 0
    ''')

In [143]:
!man qsort | head -n 10

QSORT(3)                   Linux Programmer's Manual                  QSORT(3)

NAME
       qsort, qsort_r - sort an array

SYNOPSIS
       #include <stdlib.h>

       void qsort(void *base, size_t nmemb, size_t size,
                  int (*compar)(const void *, const void *));


In [144]:
%%cpp main.c
%run gcc -m64 -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe < 001.in

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdlib.h>

//        void qsort(void *base, size_t nmemb, size_t size,
//                   int (*compar)(const void *, const void *));

int int_compare(const int* x, const int* y) {
    return *x - *y;
}

int main() {
    int n = 0;
    scanf("%d", &n);
    int* a = calloc(n, sizeof(int));
    for (int i = 0; i < n; ++i) {
        scanf("%d", &a[i]);
    }
    qsort(a, n, sizeof(int), (int(*)(const void *, const void *))int_compare);
    for (int i = 0; i < n; ++i) {
        printf("a[%d] = %d, ", i, a[i]);
    }
    printf("\n");
    free(a);
    return 0;
}

Run: `gcc -m64 -std=c99 -Wall -Werror main.c -o a.exe`

Run: `./a.exe < 001.in`

a[0] = 0, a[1] = 1, a[2] = 5, a[3] = 9, 


In [145]:
%%cpp main.c
%run gcc -DDEBUG -m64 -std=c99 -Wall -Werror main.c -o a.exe
%run ./a.exe

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdlib.h>

#define SWAP(a, b) { __typeof__(a) c = (a); (a) = (b); (b) = (c); }


void my_sort(int* a, int length) {
    // my_sort_2(a, a + length);
    for (int i = 0; i < length - 1; ++i) {
        for (int j = i + 1; j < length; ++j) {
            if (a[i] > a[j]) {
                SWAP(a[i], a[j]);
            }
        }
    }
}

void my_sort_2(int* begin, int* end) {
    // my_sort(begin, end - begin);
    for (int* i = begin; i != end; ++i) {
        for (int* j = i + 1; j != end; j++) {
            if (*i > *j) {
                SWAP(*i, *j);
            }
        }
    }
}

int main() {
    #if defined(DEBUG)
    freopen("001.in", "rt", stdin);
    #endif
    int n = 0;
    scanf("%d", &n);
    int* a = calloc(n, sizeof(int));
    for (int i = 0; i < n; ++i) {
        scanf("%d", &a[i]);
    }
    my_sort_2(a, a + n);
    for (int i = 0; i < n; ++i) {
        printf("a[%d] = %d, ", i, a[i]);
    }
    printf("\n");
    free(a);
    return 0;
}

Run: `gcc -DDEBUG -m64 -std=c99 -Wall -Werror main.c -o a.exe`

Run: `./a.exe`

a[0] = 0, a[1] = 1, a[2] = 5, a[3] = 9, 


# Строки

In [10]:
with open("001.in", "w") as f:
    f.write('''\
push 3
pop
exit
hello 6
    ''')

In [15]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe < 001.in

#include <stdio.h>
#include <string.h>

int main() {
    char str[100];
    while (1) {
        int ret = scanf("%s", str);
        if (ret <= 0) {
            break;
        }
        
        printf("READ STRING: %s\n", str);
        if (strcmp(str, "push") == 0) {
            int value;
            scanf("%d", &value);
            printf("   IT IS PUSH %d\n", value);
        } else if (strcmp(str, "hello") == 0) {
            int value;
            scanf("%d", &value);
            printf("   IT IS HELLO %d\n", value);
        }    
    }
    return 0;
}

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe < 001.in`

READ STRING: 3
READ STRING: 1
READ STRING: 2
READ STRING: 3


In [44]:
%%cpp main.c
%run gcc -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe < 001.in

#include <stdio.h>
#include <string.h>
#include <assert.h>
#include <stdbool.h> // true, false

int main() {
    char buff[100];
 
    while (true) {
        char* line = fgets(buff, sizeof(buff), stdin);
        if (!line) {
            break;
        }
        
        printf("READ LINE: '%s'\n", line);
        
        char str[100];
        int value = 0;
        int ret = sscanf(line, "%s %d", str, &value);
        if (ret <= 0) {
            printf("  ERROR\n");
        } else if (ret == 1) {
            printf("  GET ONLY STRING\n");
        }
           
        if (strcmp(str, "push") == 0) {
            assert(ret == 2);
            printf("   IT IS PUSH %d\n", value);
        } else if (strcmp(str, "hello") == 0) {
            assert(ret == 2);
            printf("   IT IS HELLO %d\n", value);
        }    
    }
    return 0;
}

Run: `gcc -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe < 001.in`

READ LINE: 'push 3
'
   IT IS PUSH 3
READ LINE: 'pop
'
  GET ONLY STRING
READ LINE: 'exit
'
  GET ONLY STRING
READ LINE: 'hello 6
'
   IT IS HELLO 6
READ LINE: '    '
  ERROR
a.exe: main.c:34: main: Assertion `ret == 2' failed.
Aborted (core dumped)


In [42]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe < 001.in

#include <stdio.h>
#include <string.h>
#include <assert.h>

int main() {
    char* a = "Hello";
    char* b = "students!";
    printf("%p\n", a);
    printf("%s %s\n", a, b);
    printf("a[1] = '%c'\n", a[1]);
    
    char c1[1000];
    int i = 0;
    for (int j = 0; a[j]; ++j) 
        c1[i++] = a[j];
    for (int j = 0; b[j]; ++j) 
        c1[i++] = b[j];
    c1[i] = '\0';
    printf("c1 = a + b = %s\n", c1);
    
    char c2[1000];
    snprintf(c2, sizeof(c2), "%s %s", a, b);
    printf("c2 = a + ' ' + b = %s\n", c2);
    
    return 0;
}

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe < 001.in`

0x4d6c60
Hello students!
a[1] = 'e'
c1 = a + b = Hellostudents!
c2 = a + ' ' + b = Hello students!


In [51]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <string.h>
#include <assert.h>


void to_upper(char* src, char* dst) {
    int i = 0;
    for (; src[i]; ++i) {
        if ('a' <= src[i] && src[i] <= 'z') {
            dst[i] = src[i] - 'a' + 'A';
        } else {
            dst[i] = src[i];
        }
    }
    dst[i] = '\0';
}

int main() {
    char* a = "Hello";
      
    char buffer[100];
    to_upper(a, buffer);
    
    printf("%s\n", buffer);
    printf("'A' = %d, 'a' = %d\n", (int)'A', (int)'a');
    
    return 0;
}

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

HELLO
'A' = 65, 'a' = 97


No manual entry for upper_case


In [62]:
%%cpp test.h

void test() {
    stack_t stack;
    init_stack(&stack);
    push(&stack, 1);
    push(&stack, 2);
    push(&stack, 3);
    push(&stack, 4);
    
    assert(top(&stack) == 4);
    pop(&stack);
    
    assert(top(&stack) == 3);
    pop(&stack);
    
    push(&stack, 5);
    
    assert(top(&stack) == 5);
    pop(&stack);
    assert(top(&stack) == 2);
    pop(&stack);
    assert(top(&stack) == 1);
    pop(&stack);
    
    destroy_stack(&stack);
    
    printf("SUCCESS\n");
}

int main() {
    test();
    return 0;
}

## Стек олимпиадника

In [66]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <string.h>
#include <assert.h>

typedef struct stack {
    int a[100500];
    int sz;
} stack_t;


void init_stack(stack_t* stack) {
    stack->sz = 0;
}

void destroy_stack(stack_t* stack) {}

void push(stack_t* stack, int elem) {
    stack->a[stack->sz++] = elem;
}

int top(stack_t* stack) {
    return stack->a[stack->sz - 1];
}

void pop(stack_t* stack) {
    --stack->sz;
    fprintf(stderr, "POP %d (was on position %d)\n", stack->a[stack->sz], stack->sz);
}


#include "test.h"

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

POP 4 (was on position 3)
POP 3 (was on position 2)
POP 5 (was on position 2)
POP 2 (was on position 1)
POP 1 (was on position 0)
SUCCESS


## Стек странного человека

In [72]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <assert.h>

typedef struct node {
    int elem;
    struct node* previous;
} node_t;



typedef struct stack {
    node_t* top;
} stack_t;


void init_stack(stack_t* stack) {
    stack->top = NULL;
}

void pop(stack_t* stack);

void destroy_stack(stack_t* stack) {
    while (stack->top) {
        pop(stack);
    }
}

void push(stack_t* stack, int elem) {
    node_t* node = calloc(1, sizeof(node_t));
    node->elem = elem;
    node->previous = stack->top;
    stack->top = node;
}

int top(stack_t* stack) {
    return stack->top->elem;
}

void pop(stack_t* stack) {
    node_t* old_top = stack->top;
    stack->top = old_top->previous;
    fprintf(stderr, "POP %d\n", old_top->elem);
    free(old_top);
}


#include "test.h"

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

POP 4
POP 3
POP 5
POP 2
POP 1
SUCCESS


## Cтек здорового человека

In [79]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <assert.h>

typedef struct stack {
    int* a;
    int sz;
    int max_sz;
} stack_t;


void init_stack(stack_t* stack) {
    *stack = (stack_t){0};
}

void destroy_stack(stack_t* stack) {
    free(stack->a);
}

void push(stack_t* stack, int elem) {
    if (stack->sz == stack->max_sz) {
        stack->max_sz += (stack->max_sz == 0);
        stack->max_sz *= 2;
        (*stack).a = realloc(stack->a, stack->max_sz * sizeof(int));
    }
    stack->a[stack->sz++] = elem;
}

int top(stack_t* stack) {
    return stack->a[stack->sz - 1];
}

void pop(stack_t* stack) {
    --stack->sz;
    fprintf(stderr, "POP %d (was on position %d)\n", stack->a[stack->sz], stack->sz);
}


#include "test.h"

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

POP 4 (was on position 3)
POP 3 (was on position 2)
POP 5 (was on position 2)
POP 2 (was on position 1)
POP 1 (was on position 0)
SUCCESS


# ASAN

aka address-sanitizer

Опция компилятора `-fsanitize=address`

## 1) Утечки

In [83]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdlib.h>
#include <assert.h>

int main() {
    int* array = calloc(10, sizeof(int));
    assert(array[0] == 0);
    return 0;
}

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`


==2729==ERROR: LeakSanitizer: detected memory leaks

Direct leak of 40 byte(s) in 1 object(s) allocated from:
    #0 0x493ba2 in calloc (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x493ba2)
    #1 0x4c313d in main (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x4c313d)
    #2 0x7f41fe10e0b2 in __libc_start_main /build/glibc-YYA7BZ/glibc-2.31/csu/../csu/libc-start.c:308:16

SUMMARY: AddressSanitizer: 40 byte(s) leaked in 1 allocation(s).


## 2) Проезды

In [85]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdlib.h>
#include <assert.h>

int main() {
    int* array = calloc(10, sizeof(int));
    assert(array[100500] == 0);
    return 0;
}

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

AddressSanitizer:DEADLYSIGNAL
==2746==ERROR: AddressSanitizer: SEGV on unknown address 0x6040000622e0 (pc 0x0000004c3191 bp 0x7ffdea6c0450 sp 0x7ffdea6c0430 T0)
==2746==The signal is caused by a READ memory access.
    #0 0x4c3191 in main (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x4c3191)
    #1 0x7f8e489200b2 in __libc_start_main /build/glibc-YYA7BZ/glibc-2.31/csu/../csu/libc-start.c:308:16
    #2 0x41b2ed in _start (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x41b2ed)

AddressSanitizer can not provide additional info.
SUMMARY: AddressSanitizer: SEGV (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x4c3191) in main
==2746==ABORTING


## 3) Использование памяти после free

In [86]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdlib.h>
#include <assert.h>

int main() {
    int* array = calloc(10, sizeof(int));
    free(array);
    assert(array[5] == 0);
    return 0;
}

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

==2753==ERROR: AddressSanitizer: heap-use-after-free on address 0x6040000000a4 at pc 0x0000004c3199 bp 0x7ffdfa1b0400 sp 0x7ffdfa1b03f8
READ of size 4 at 0x6040000000a4 thread T0
    #0 0x4c3198 in main (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x4c3198)
    #1 0x7fdcdaa230b2 in __libc_start_main /build/glibc-YYA7BZ/glibc-2.31/csu/../csu/libc-start.c:308:16
    #2 0x41b2ed in _start (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x41b2ed)

0x6040000000a4 is located 20 bytes inside of 40-byte region [0x604000000090,0x6040000000b8)
freed by thread T0 here:
    #0 0x4937ad in free (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x4937ad)
    #1 0x4c314d in main (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x4c314d)
    #2 0x7fdcdaa230b2 in __libc_start_main /build/glibc-YYA7BZ/glibc-2.31/csu/../csu/libc-start.c:308:16

previously allocated by thread T0 here:
    #0 0x493ba2 in calloc (/home/pechatnov/vbox/caos/extra-c-basics/a.exe+0x493ba2)
    #1 0x4c313d in main (/home/pechatn

## ... и так далее

# MergeSort

In [96]:
%%cpp merge.c
%run gcc --sanitize=address merge.c -o merge.exe
%run ./merge.exe

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

typedef int elem_t;

void merge_sort_impl(elem_t* a, int size, elem_t* buff) {
    if (size <= 1) {
        return;
    }
    int lsize = size / 2;
    int rsize = size - lsize;
    
    merge_sort_impl(a, lsize, buff);
    merge_sort_impl(a + lsize, rsize, buff);
    
    int L = 0, R = lsize;
    int k = 0;
    while (L < lsize && R < size) {
        if (a[L] < a[R]) {
            buff[k++] = a[L++];
        } else {
            buff[k++] = a[R++];    
        }
    }
    while (L < lsize) {
        buff[k++] = a[L++];
    }
    while (R < size) {
        buff[k++] = a[R++];
    }
    memcpy(a, buff, size * sizeof(elem_t));
}


void merge_sort(elem_t* a, int size) {
     if (size < 10000 / sizeof(elem_t)) {
        elem_t buff[size];
        merge_sort_impl(a, size, buff);
     } else {
        elem_t* buff = calloc(size, sizeof(elem_t));
        merge_sort_impl(a, size, buff);
        free(buff);
    }
}



int main() {
    //int a[] = {1, 5, 8, 2, 4, 9, 3};
    int a[] = {1, 5, 8, 2, 4, 9, 3};
    int a_size = sizeof(a) / sizeof(a[0]);
    
    merge_sort(a, a_size);
        
    for (int i = 0; i < a_size; ++i) {
        printf("%d, ", a[i]);
    }
    printf("\n");
    return 0;
}

Run: `gcc --sanitize=address merge.c -o merge.exe`

Run: `./merge.exe`

1, 2, 3, 4, 5, 8, 9, 


In [92]:
!man calloc

MALLOC(3)                  Linux Programmer's Manual                 MALLOC(3)

NAME
       malloc, free, calloc, realloc - allocate and free dynamic memory

SYNOPSIS
       #include <stdlib.h>

       void *malloc(size_t size);
       void free(void *ptr);
       void *calloc(size_t nmemb, size_t size);
       void *realloc(void *ptr, size_t size);
       void *reallocarray(void *ptr, size_t nmemb, size_t size);

   Feature Test Macro Requirements for glibc (see feature_test_macros(7)):

       reallocarray():
           Since glibc 2.29:
               _DEFAULT_SOURCE
           Glibc 2.28 and earlier:
               _GNU_SOURCE

DESCRIPTION
       The malloc() function allocates size bytes and returns a pointer to the
       allocated memory.  The memory is not initialized.  If size is  0,  then
       malloc()  returns either NULL, or a unique pointer value that can later
       be successfully passed to free().

       The free() function frees the memory space pointed  to  by  pt

# Queue

In [101]:
%%cpp test.h

void test() {
    queue_t q;
    init_queue(&q);
    push(&q, 1);
    push(&q, 2);
    push(&q, 3);
    push(&q, 4);
    
    assert(front(&q) == 1);
    pop_front(&q);

    assert(front(&q) == 2);
    pop_front(&q);

    assert(front(&q) == 3);
    pop_front(&q);
    
    push(&q, 5);
    
    assert(front(&q) == 4);
    pop_front(&q);

    push(&q, 6);
    
    assert(front(&q) == 5);
    pop_front(&q);
    
    destroy_queue(&q);
    
    printf("SUCCESS\n");
}

int main() {
    test();
    return 0;
}

# Очередь олимпиадника

In [102]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <string.h>
#include <assert.h>

typedef struct queue {
    int a[100500];
    int first;
    int last; // не включительно
} queue_t;


void init_queue(queue_t* q) {
    q->first = q->last = 0;
}

void destroy_queue(queue_t* q) {}

void push(queue_t* q, int elem) {
    q->a[q->last++] = elem;
}

int front(queue_t* q) {
    return q->a[q->first];
}

void pop_front(queue_t* q) {
    ++q->first;
    fprintf(stderr, "POP %d (was on position %d)\n", q->a[q->first], q->first);
}


#include "test.h"

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

POP 2 (was on position 1)
POP 3 (was on position 2)
POP 4 (was on position 3)
POP 5 (was on position 4)
POP 6 (was on position 5)
SUCCESS


# Очередь человека любящего указатели 

In [107]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <assert.h>

typedef struct node {
    int elem;
    struct node* next;
} node_t;

typedef struct queue {
    node_t* first;
    node_t* last; // включительно
} queue_t;


void pop_front(queue_t* q);

void init_queue(queue_t* q) {
    q->first = NULL;
}

void destroy_queue(queue_t* q) {
    while (q->first) {
        pop_front(q);
    }
}

void push(queue_t* q, int elem) {
    node_t* new_node = calloc(1, sizeof(node_t));
    new_node->elem = elem;
    
    if (q->first) {
        q->last->next = new_node;
    } else {
        q->first = new_node;
    }
    q->last = new_node;
}

int front(queue_t* q) {
    return q->first->elem;
}

void pop_front(queue_t* q) {
    node_t* x = q->first;
    q->first = q->first->next;
    fprintf(stderr, "POP %d\n", x->elem);
    free(x);
}


#include "test.h"

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

POP 1
POP 2
POP 3
POP 4
POP 5
POP 6
SUCCESS


# Циклическая очередь без расширения

In [108]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <string.h>
#include <assert.h>

typedef struct queue {
    int a[1024];
    int first;
    int last; // не включительно
} queue_t;


void init_queue(queue_t* q) {
    q->first = q->last = 0;
}

void destroy_queue(queue_t* q) {}

void push(queue_t* q, int elem) {
    q->a[(q->last++) % 1024] = elem;
}

int front(queue_t* q) {
    return q->a[q->first % 1024];
}

void pop_front(queue_t* q) {
    ++q->first;
    fprintf(stderr, "POP %d (was on position %d)\n", q->a[q->first % 1024], q->first);
}


#include "test.h"

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

POP 2 (was on position 1)
POP 3 (was on position 2)
POP 4 (was on position 3)
POP 5 (was on position 4)
POP 6 (was on position 5)
SUCCESS


In [4]:
%%cpp main.c
d

# Тестирование алгоритма (пример с семинара)

In [46]:
%%cpp main.c
%run gcc -std=c99 -Wall -Werror -fsanitize=address main.c -o unused.exe
%run gcc -DDO_TEST -std=c99 -Wall -Werror -fsanitize=address main.c -o o.exe
%run ./o.exe 

#define _XOPEN_SOURCE 700

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <string.h>
#include <stdbool.h>
#include <time.h>
#include <stdint.h>
#include <inttypes.h>


typedef int elem_t;

void slow_sort(elem_t* first, elem_t* last) {
    for (elem_t* i = first; i != last; ++i) {
        for (elem_t* j = i + 1; j != last; ++j) {
            if (*i > *j) {
                elem_t tmp = *i;
                *i = *j;
                *j = tmp;
            }
        }
    }
}

//#define fprintf(...) ;

void quick_sort(elem_t* first, elem_t* last) {
    if (last - first < 2) {
        return;
    }

    // for (elem_t* i = first; i != last; ++i) {
    //     fprintf(stderr, "%d ", *i);
    // }
    // fprintf(stderr, "\n");

    elem_t pivot = *(first + (last - first) / 2);
    // fprintf(stderr, "pivot = %d\n", pivot);

    elem_t* L = first;
    elem_t* R = last - 1;

    while (L <= R) {
        while (*L < pivot && L <= R) ++L;
        while (*R > pivot && L <= R) --R;
        if (L <= R) {
            elem_t tmp = *L;
            *L = *R;
            *R = tmp;
            ++L;
            --R;
        }
    }


    // fprintf(stderr, "sub sort L %d-%d\n", 0, R + 1 - first);
    // for (elem_t* i = first; i != R + 1; ++i) {
    //     fprintf(stderr, "%d ", *i);
    // }
    // fprintf(stderr, "\n");
    quick_sort(first, R + 1);
    // fprintf(stderr, "sub sort R %d-%d\n", L - first, last - first);
    // for (elem_t* i = L; i != last; ++i) {
    //     fprintf(stderr, "%d ", *i);
    // }
    // fprintf(stderr, "\n");
    quick_sort(L, last);
}

#ifdef DO_TEST

uint64_t get_ms() {
    struct timespec spec;
    clock_gettime(CLOCK_REALTIME, &spec);
    return (uint64_t)spec.tv_sec * 1000 + spec.tv_nsec / 1.0e6;
}

void test() {
    {
        int a[] = {3, 2, 1};
        slow_sort(a, a + sizeof(a) / sizeof(int));
        assert(a[0] == 1);
        assert(a[1] == 2);
        assert(a[2] == 3);
    }
    fprintf(stderr, "success 1\n"); fflush(stderr);
    {
        int a[] = {3};
        slow_sort(a, a + sizeof(a) / sizeof(int));
        assert(a[0] == 3);
    }
    fprintf(stderr, "success 2\n"); fflush(stderr);
    {
        int a[] = {1};
        slow_sort(a, a);
    }
    fprintf(stderr, "success 3\n"); fflush(stderr);
    for (int counter = 0; counter < 1000; counter++) {
        // fprintf(stderr, "%d =================== \n", counter);
        int n = 1 + rand() % 4;
        int a[n];
        for (int i = 0; i < n; ++i) {
            a[i] = rand() % 10;
        }
        int b[n];
        int c[n];
        memcpy(b, a, n * sizeof(int));
        memcpy(c, a, n * sizeof(int));

        slow_sort(b, b + n);
        quick_sort(c, c + n);

        bool good = true;
        for (int i = 0; i < n; ++i) {
            if (b[i] != c[i]) {
                good = false;
            }
        }
        if (!good) {
            printf("test: ");
            for (int i = 0; i < n; ++i) printf("%d ", a[i]);
            printf("\n");
            printf("expected: ");
            for (int i = 0; i < n; ++i) printf("%d ", b[i]);
            printf("\n");
            printf("actual: ");
            for (int i = 0; i < n; ++i) printf("%d ", c[i]);
            printf("\n");
            break;
        }
    }
    fprintf(stderr, "success 4\n"); fflush(stderr);
    {
        int n = 1000000;
        int* a = calloc(n, sizeof(int));
        for (int i = 0; i < n; ++i) {
            a[i] = rand() % n;
        }
        uint64_t start = get_ms();
        quick_sort(a, a + n);
        printf("Long sort: %" PRIu64 " ms\n", get_ms() - start);
        for (int i = 0; i + 1 < n; ++i) {
            assert(a[i] <= a[i + 1]);
        }
        free(a);
    }

    {
        int n = 1000000;
        int* a = calloc(n, sizeof(int));
        for (int i = 0; i < n; ++i) {
            a[i] = i;
        }
        uint64_t start = get_ms();
        quick_sort(a, a + n);
        printf("Already sorted long sort: %" PRIu64 " ms\n", get_ms() - start);
        for (int i = 0; i + 1 < n; ++i) {
            assert(a[i] <= a[i + 1]);
        }
        free(a);
    }

    {
        int n = 1000000;
        int* a = calloc(n, sizeof(int));
        for (int i = 0; i < n; ++i) {
            a[i] = 0;
        }
        uint64_t start = get_ms();
        quick_sort(a, a + n);
        printf("Zero long sort: %" PRIu64 " ms\n", get_ms() - start);
        for (int i = 0; i + 1 < n; ++i) {
            assert(a[i] <= a[i + 1]);
        }
        free(a);
    }

    {
        int n = 1000000;
        int* a = calloc(n, sizeof(int));
        for (int i = 0; i < n; ++i) {
            a[i] = n - i;
        }
        uint64_t start = get_ms();
        quick_sort(a, a + n);
        printf("Descending sorted long sort: %" PRIu64 " ms\n", get_ms() - start);
        for (int i = 0; i + 1 < n; ++i) {
            assert(a[i] <= a[i + 1]);
        }
        free(a);
    }
}
#endif

int main() {
#ifdef DO_TEST
    test();
    return 0;
#endif

    int n;
    scanf("%d", &n);
    int* a = calloc(n, sizeof(int));
    for (int i = 0; i < n; ++i) {
        scanf("%d", &a[i]);
    }
    quick_sort(a, a + n);

    for (int i = 0; i < n; ++i) {
        printf("%d ", a[i]);
    }
    free(a);
    return 0;
}


Run: `gcc -std=c99 -Wall -Werror -fsanitize=address main.c -o unused.exe`

Run: `gcc -DDO_TEST -std=c99 -Wall -Werror -fsanitize=address main.c -o o.exe`

Run: `./o.exe`

success 1
success 2
success 3
success 4
Long sort: 1845 ms
Already sorted long sort: 456 ms
Zero long sort: 650 ms
Descending sorted long sort: 506 ms


In [9]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <string.h>
#include <assert.h>

#define print_int(x) printf("%s = %d\n", #x, (x))

int main() {
    int a = 42;
    print_int(a);
    print_int(a++);
    print_int(a);
    
    int b = 42;
    print_int(b);
    print_int(++b);
    print_int(b);
}

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

a = 42
a++ = 42
a = 43
b = 42
++b = 43
b = 43


In [24]:
%%cpp main.c
%run clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <string.h>
#include <assert.h>

#define print_int(x) printf("%s = %d\n", #x, (x))

int main() {
    int d[] = {1, 10, 16, 26, 9, 6};
    for (int i = 0; i < sizeof(d) / sizeof(int); ++i) {
        char c = (d[i] < 10) ? ('0' + d[i]) : ('A' + d[i] - 10);
        printf("%c", c);
    }
    printf("\n");
}

Run: `clang -std=c99 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

1AGQ96


In [34]:
%%cpp main.c
%run clang -std=c11 -Wall -Werror -fsanitize=address main.c -o a.exe
%run ./a.exe 

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <assert.h>


int main() {
//     int n;
//     scanf("%d", &n);
//     int d[n];
//     (void)d;

    int* a = (int*)calloc(3, sizeof(int));
    free(a);
    return 0;
}

Run: `clang -std=c11 -Wall -Werror -fsanitize=address main.c -o a.exe`

Run: `./a.exe`

^C


In [53]:
%%cpp merge.c
%run gcc --sanitize=address merge.c -o merge.exe
%run ./merge.exe

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

#define swap(a, b) { __typeof__(a) c = (a); (a) = (b); (b) = (c); }

#define mul(a, b) ((a) * (b))

int main() {
    printf("%d\n", mul(1 + 1, 1 + 1));
    int a = 40, b = 50;
    printf("%d %d\n", a, b);
    swap(a, b);
    printf("%d %d\n", a, b);
    return 0;
}

Run: `gcc --sanitize=address merge.c -o merge.exe`

Run: `./merge.exe`

4
40 50
50 40


In [56]:
%%cpp merge.c
%run gcc --sanitize=address merge.c -o merge.exe
%run ./merge.exe

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

#define swap(a, b) { __typeof__(a) c = (a); (a) = (b); (b) = (c); }

#define mul(a, b) ((a) * (b))

int main() {
    //n (1 + 2 / 2 + 3 / 4 + 4 / 8 + 5 / 16)
    double c = 0;
    double mul = 1.0;
    for (int i = 1; i < 10; ++i) {
        c += i / mul;
        mul *= 2;
        printf("%lf\n", c);
    }
    return 0;
}

Run: `gcc --sanitize=address merge.c -o merge.exe`

Run: `./merge.exe`

1.000000
2.000000
2.750000
3.250000
3.562500
3.750000
3.859375
3.921875
3.957031


# Задача на код на Си и совсем капельку на алгоритмы

In [97]:
%%cpp task.c
%run gcc -fsanitize=address task.c -o task.exe
%run ./task.exe

#include <stdio.h>
#include <string.h>
#include <assert.h>


typedef struct {
    int x, y;
} point_t;

// 1 пункт
int compare_struct(point_t* a, point_t* b) {
    // напишите компаратор такой же, какой нужен для qsort
    // используйте man qsort в консоли или поисковике
    if (a->x != b->x) 
        return a->x - b->x;
    return a->y - b->y;
}

// 2 пункт
void quadratic_sort(
    void* base, size_t array_size, size_t elem_size, 
    int (*comparator)(const void *, const void *)
) {
    // напишите квадратичную сортировку
    char* base_c = (char*)base;
    char tmp_buff[elem_size];
    for (int i = 0; i < array_size; ++i) {
        for (int j = i + 1; j < array_size; ++j) {
            char* a = base + i * elem_size;
            char* b = base + j * elem_size;
            if (comparator(a, b) > 0) {
                memcpy(tmp_buff, a, elem_size);
                memcpy(a, b, elem_size);
                memcpy(b, tmp_buff, elem_size);
            }
        }
    }
}

// 3 пункт
void do_test_1() {
    // напишите тесты на quadratic_sort с использованием структуры point_t
    {
        point_t arr[2] = {{3, 4}, {1, 2}};
        quadratic_sort(arr, sizeof(arr) / sizeof(point_t), sizeof(point_t), 
                       (int (*)(const void *, const void *))compare_struct);
        assert(arr[0].x == 1);
        assert(arr[0].y == 2);
        assert(arr[1].x == 3);
        assert(arr[1].y == 4);
    }
    {
        point_t arr[] = {{3, 4}, {1, 2}, {-1, -3}, {2, 10}, {-1, -5}};
        quadratic_sort(arr, sizeof(arr) / sizeof(point_t), sizeof(point_t), 
                       (int (*)(const void *, const void *))compare_struct);
        assert(arr[0].x == -1);
        assert(arr[0].y == -5);
        assert(arr[1].x == -1);
        assert(arr[1].y == -3);
        assert(arr[2].x == 1);
        assert(arr[2].y == 2);
        
        assert(arr[3].x == 2);
        assert(arr[3].y == 10);
        
        assert(arr[4].x == 3);
        assert(arr[4].y == 4);
    }
}


// 4 пункт
// напишите макрос, который будет создавать функцию сортировки для стандартных типов
// (использовать обычный < для сравнения)
// при этом делегировать сортировку функции quadratic_sort
#define DECLARE_SORT_FUNCTION_IMPL_2(name, type, tag)                                     \
    int compare_##tag(type* a, type* b) {                                                 \
        return (*a < *b) ? -1 : ((*a > *b) ? 1 : 0);                                      \
    }                                                                                     \
    void name(type* first, type* last) {                                                  \
        quadratic_sort(first, last - first, sizeof(*first),                               \
            (int (*)(const void *, const void *))compare_##tag);                          \
    }

#define DECLARE_SORT_FUNCTION_IMPL(name, type, tag) DECLARE_SORT_FUNCTION_IMPL_2(name, type, tag)
#define DECLARE_SORT_FUNCTION(name, type) DECLARE_SORT_FUNCTION_IMPL(name, type, __LINE__)


DECLARE_SORT_FUNCTION(sort_int, int);
DECLARE_SORT_FUNCTION(sort_double, double);

// 5 пункт
void do_test_2() {
    // протестируйте, что функция sort_int правильно работает
    {
        int a[] = {1, 3, 2, 4};
        sort_int(a, a + sizeof(a) / sizeof(int));
        assert(a[0] == 1);
        assert(a[1] == 2);
        assert(a[2] == 3);
        assert(a[3] == 4);
    }
    {
        int a[] = {3, 1};
        sort_int(a, a + sizeof(a) / sizeof(int));
        assert(a[0] == 1);
        assert(a[1] == 3);
    }
    
    {
        double a[] = {1, 3, 2, 4};
        sort_double(a, a + sizeof(a) / sizeof(double));
        assert(a[0] == 1);
        assert(a[1] == 2);
        assert(a[2] == 3);
        assert(a[3] == 4);
    }
    {
        double a[] = {3, 1};
        sort_double(a, a + sizeof(a) / sizeof(double));
        assert(a[0] == 1);
        assert(a[1] == 3);
    }
}


int main() {
    //do_test_1();
    do_test_2();
    fprintf(stderr, "SUCCESS\n");
    return 0;
}







Run: `gcc -fsanitize=address task.c -o task.exe`

Run: `./task.exe`

SUCCESS


In [92]:
!man qsort

QSORT(3)                   Linux Programmer's Manual                  QSORT(3)

NNAAMMEE
       qsort, qsort_r - sort an array

SSYYNNOOPPSSIISS
       ##iinncclluuddee <<ssttddlliibb..hh>>

       vvooiidd qqssoorrtt((vvooiidd **_b_a_s_e,, ssiizzee__tt _n_m_e_m_b,, ssiizzee__tt _s_i_z_e,,
                  iinntt ((**_c_o_m_p_a_r))((ccoonnsstt vvooiidd **,, ccoonnsstt vvooiidd **))));;

       vvooiidd qqssoorrtt__rr((vvooiidd **_b_a_s_e,, ssiizzee__tt _n_m_e_m_b,, ssiizzee__tt _s_i_z_e,,
                  iinntt ((**_c_o_m_p_a_r))((ccoonnsstt vvooiidd **,, ccoonnsstt vvooiidd **,, vvooiidd **)),,
                  vvooiidd **_a_r_g));;

   Feature Test Macro Requirements for glibc (see ffeeaattuurree__tteesstt__mmaaccrrooss(7)):

       qqssoorrtt__rr(): 

In [ ]:
%%cpp merge.c
%run gcc --sanitize=address merge.c -o merge.exe
%run ./merge.exe

#include <stdio.h>
#include <string.h>
#include <stdlib.h>


typedef struct {
    int sum;
    int i, j;
} result_t;

int main() {
    int a[], b[];
    
    int max_a = a[0];
    int pos_i = 0;
    
    result_t result = {.sum = a[0] + b[0], .i = 0, .j = 0};
        
    for (int j = 0; i < N; ++j) {
        if (a[i] > max_a) {
            max_a = a[j];
            pos_i = j;
        }
        result_t candidate = {.sum = max_a + b[j], .i = pos_i, .j = j};
        if (candidate.sum > result.sum) {
            result = candidate;
        }
    }
    
    print("%d %d %d", result.sum, result.i, result.j);
    
    
    
    return 0;
}

In [ ]:
%%cpp merge.c
%run gcc --sanitize=address merge.c -o merge.exe
%run ./merge.exe

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

#define SIZE


int f(int* a)
    
int f(int** a)
   
    
    
int* very_very_bad_g(int n) {
    int arr[n];
    for (int i = 0; i < n; ++i) {
        arr[i] = i * i;
    }
    return arr;
}

    
int* g(int n) {
    int* arr = calloc(n, sizeof(int));
    for (int i = 0; i < n; ++i) {
        arr[i] = i * i;
    }
    return arr;
}

void g(int n, int* arr) {
    for (int i = 0; i < n; ++i) {
        arr[i] = i * i;
    }
}

int main() {
    N x M
    
    int* a = calloc(N * M, sizeof(int));
    
    // NxM: 3x2
    // 12
    // 23
    // 45
    
    0 <= i < N
    0 <= j < M
    a[i][j] -> a[i * M + j] // max (N - 1) * M + (M - 1) = N * M - 1
    free(a);

    
    int** a = calloc(N, sizeof(int*));
    for (int i = 0; i < N; ++i) {
        a[i] = calloc(M, sizeof(int));
    }
    a[i][j]
    
    for (int i = 0; i < N; ++i) {
        free(a[i]);
    }
    free(a);
    
    
    int* a = calloc(N, sizeof(int));
    ...
    int* b = calloc(N, sizeof(int));
    for (int i = 0; i < N; ++i) {
        a[i] = b[i];
    }
    memcpy(a, b, N * sizeof(int));
    free(a);
    free(b);
    
    
    return 0;
}


In [ ]:
int x = (y*z)%

In [17]:
%%cpp merge.c
%run gcc --sanitize=address merge.c -o merge.exe
%run ./merge.exe

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

int f() {
    return 1;
}

typedef struct {} struct_t;

int main() {
    int a = 1134234243;
    int b = 134523432;
    ((int64_t)a * (int64_t)b);
   
    
    return 0;
}

Run: `gcc --sanitize=address merge.c -o merge.exe`

Run: `./merge.exe`

$$sum_{i \in 0..N} (10^i \cdot a_i)$$

In [46]:
%%cpp merge.c
%run gcc -Wall -Werror --sanitize=address merge.c -o merge.exe
%run ./merge.exe

#include <stdio.h>
#include <string.h>
#include <stdlib.h>

#define MODULE 10

typedef struct {
    int len;
    int* digits;
} big_number_t;

big_number_t to_big_number(char* s) {
    // "1234" aka {'1', '2', '3', '4'}
    // ->
    // {4, 3, 2, 1}
    // real value = 4 * 10**0 + 3 * 10**1 + 2 * 10**2 + 1 * 10**3
    big_number_t result;
    result.len = strlen(s);
    result.digits = calloc(result.len, sizeof(int));
    for (int i = 0; i < result.len; ++i) {
        result.digits[i] = s[result.len - 1 - i] - '0'; // '0'..'9' - '0' ~ 0..9
    }
    return result;
}

void destroy_big_number(big_number_t n) {
    free(n.digits);
    n.digits = NULL;
}

big_number_t add_big_number(big_number_t a, big_number_t b) {
    if (a.len < b.len) {
        return add_big_number(b, a);
    }
    big_number_t c;
    c.len = a.len + 1;
    c.digits = calloc(c.len, sizeof(int));
    for (int i = 0; i < b.len; ++i) {
        c.digits[i] += a.digits[i] + b.digits[i];
        c.digits[i + 1] += c.digits[i] / MODULE;
        c.digits[i] %= MODULE;
    }
    for (int i = b.len; i < a.len; ++i) {
        c.digits[i] += a.digits[i];
        c.digits[i + 1] += c.digits[i] / MODULE;
        c.digits[i] %= MODULE;
    }
    while (c.len && c.digits[c.len - 1] == 0) {
        --c.len;
    }
    return c;
}

big_number_t mul_big_number(big_number_t a, big_number_t b) {
    big_number_t c;
    c.len = a.len + b.len;
    c.digits = calloc(c.len, sizeof(int));
    for (int i = 0; i < a.len; ++i) {      
        for (int j = 0; j < b.len; ++j) {
            c.digits[i + j] += a.digits[i] * b.digits[j];
        }
    }
    for (int i = 0; i + 1 < c.len; ++i) {
        c.digits[i + 1] += c.digits[i] / MODULE;
        c.digits[i] %= MODULE;
    }
    while (c.len && c.digits[c.len - 1] == 0) {
        --c.len;
    }
    return c;
}

big_number_t div_small_big_number(big_number_t a, int b, int* remainder) {
    big_number_t c;
    c.len = a.len;
    c.digits = calloc(c.len, sizeof(int));
    int r = 0;
    for (int i = a.len - 1; i >= 0; --i) {
        int cur = r * MODULE + a.digits[i];
        c.digits[i] = cur / b;
        r = cur % b;
    }
    if (remainder) {
        *remainder = r;
    }
    while (c.len && c.digits[c.len - 1] == 0) {
        --c.len;
    }
    return c;
}

void print_big_number(big_number_t n) {
    for (int i = n.len - 1; i >= 0; --i) {
        printf("%c", '0' + n.digits[i]);
    }
}


int main() {
    char* a = "999999999999999999";
    char* b = "999999999999999999";
    big_number_t a_n = to_big_number(a);
    big_number_t b_n = to_big_number(b);
    big_number_t c_n = add_big_number(a_n, b_n);
    big_number_t d_n = div_small_big_number(c_n, 11, NULL);
    print_big_number(a_n); printf("\n");
    print_big_number(b_n); printf("\n");
    print_big_number(c_n); printf("\n");
    print_big_number(d_n); printf("\n");
    
    destroy_big_number(a_n); 
    destroy_big_number(c_n); 
    destroy_big_number(b_n); 
    destroy_big_number(d_n); 
    
    return 0;
}

Run: `gcc -Wall -Werror --sanitize=address merge.c -o merge.exe`

Run: `./merge.exe`

999999999999999999
999999999999999999
1999999999999999998
181818181818181818


In [17]:
%%sql
SELECT a from B;

UsageError: Cell magic `%%sql` not found.
